Previous notebooks

In [9]:
%run reference_circuit.ipynb

Libraries

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

Configuration

In [3]:
NUM_QUBITS = 3
POP_SIZE = 10
N_GEN = 50
MUTATION_RATE = 0.5
ELITE_SIZE = 2
LAMBDA_DEPTH = 0.1

# Target distribution
x = np.arange(2**NUM_QUBITS)
p_target = np.exp(-0.5 * ((x - 3.5) / 1.0)**2)
p_target /= np.sum(p_target)
psi_target = np.sqrt(p_target)

Helper functions

In [26]:
def kl_divergence(p, q, eps=1e-10):
    p = np.clip(p, eps, 1)
    q = np.clip(q, eps, 1)
    return np.sum(p * np.log(p / q))

GATES = ['h', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'cx', 'cz', 'sx']

def random_gate():
    g = random.choice(GATES)
    if g in ['rx', 'ry', 'rz']:
        return (g, random.randint(0, NUM_QUBITS - 1), random.uniform(0, 2*np.pi))
    elif g == 'cx':
        control = random.randint(0, NUM_QUBITS - 1)
        target = (control + 1) % NUM_QUBITS
        return (g, control, target)
    else:
        return (g, random.randint(0, NUM_QUBITS - 1))

def build_circuit(gates):
    qc = QuantumCircuit(NUM_QUBITS)
    for gate in gates:
        if gate[0] in ['rx', 'ry', 'rz']:
            getattr(qc, gate[0])(gate[2], gate[1])
        elif gate[0] == 'cx':
            qc.cx(gate[1], gate[2])
        else:
            getattr(qc, gate[0])(gate[1])
    return qc
    #GHZ = np.array([1, 0, 0, 0, 0, 0, 0, 1]) / np.sqrt(2)
    #GHZ_evolve_matrix = householder_unitary(GHZ)
    #qc_ghz = create_reference_circuit(GHZ_evolve_matrix)

    #return qc_ghz

def fitness(ind):
    qc = build_circuit(ind)
    state = Statevector.from_instruction(qc)
    probs = np.abs(state.data)**2
    kl = kl_divergence(p_target, probs)
    penalty = LAMBDA_DEPTH * qc.depth()
    return -(kl + penalty)  # maximize negative loss

# GA functions
def mutate(ind):
    ind = ind.copy()
    if random.random() < 0.5 and len(ind) > 1:
        ind[random.randint(0, len(ind)-1)] = random_gate()
    else:
        ind.append(random_gate())
    return ind

def crossover(p1, p2):
    if len(p1) < 2 or len(p2) < 2:
        return p1
    point = random.randint(1, min(len(p1), len(p2)) - 1)
    return p1[:point] + p2[point:]
    
def softmax(x, temperature=0.5): # higher temperature makes the distribution more flat (more uniform selection)
    # recomended temperature [0.1, 0.8]
    x = np.array(x)
    x = x - np.max(x)  # For numerical stability
    exps = np.exp(x / temperature)
    return exps / np.sum(exps)

def select(pop, fitnesses, temperature=0.5):
    probs = softmax(fitnesses, temperature)
    selected = random.choices(pop, weights=probs, k=POP_SIZE - ELITE_SIZE)
    # Keep elites deterministically
    elite_indices = np.argsort(fitnesses)[-ELITE_SIZE:]
    elites = [pop[i] for i in elite_indices]
    return elites + selected

def random_individual(length=10):
    return [random_gate() for _ in range(length)]

GA loop

In [29]:
population = [random_individual() for _ in range(POP_SIZE)]
print(population)
#print(population)

GHZ = np.array([1, 0, 0, 0, 0, 0, 0, 1]) / np.sqrt(2)
GHZ_evolve_matrix = householder_unitary(GHZ)
qc_ghz = create_reference_circuit(GHZ_evolve_matrix)

#print(build_circuit(GATES))
#print(qc_ghz)

for gate in qc_ghz.data:
    #print(gate)
    population.append([gate[0].name, gate[1][0], gate[1][1] if len(gate[1]) > 1 else None])

print(population)

[[('z', 1), ('cz', 0), ('y', 1), ('x', 0), ('z', 1), ('x', 2), ('cz', 1), ('ry', 1, 4.414576179208218), ('cx', 1, 2), ('rz', 2, 0.8611612488023543)], [('ry', 2, 0.08334990438132531), ('h', 0), ('rz', 2, 2.3314480074787416), ('z', 0), ('h', 2), ('cz', 0), ('cz', 0), ('z', 0), ('cz', 1), ('sx', 2)], [('x', 1), ('y', 2), ('sx', 0), ('cz', 2), ('cx', 2, 0), ('rz', 2, 1.9160843786457153), ('x', 0), ('h', 0), ('ry', 2, 2.7619440845148313), ('h', 0)], [('rz', 1, 1.8871151500089591), ('z', 1), ('rz', 0, 0.8320995995845127), ('sx', 0), ('y', 0), ('x', 1), ('h', 0), ('x', 1), ('h', 0), ('rx', 0, 6.120122513450943)], [('rx', 2, 6.082520645080442), ('cx', 2, 0), ('y', 0), ('h', 2), ('y', 1), ('rz', 0, 3.39639832881497), ('sx', 1), ('rx', 1, 0.22475577005323014), ('cz', 0), ('y', 1)], [('cz', 1), ('cx', 1, 2), ('rz', 0, 1.5715861088630074), ('sx', 1), ('x', 2), ('ry', 1, 2.4068068383753496), ('y', 0), ('cz', 1), ('ry', 0, 5.870579823310866), ('cx', 1, 2)], [('z', 2), ('cz', 1), ('y', 2), ('rx', 1, 

C:\Users\jiriv\AppData\Local\Temp\ipykernel_27632\2823086221.py:14: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  population.append([gate[0].name, gate[1][0], gate[1][1] if len(gate[1]) > 1 else None])


In [28]:
#population = [random_individual() for _ in range(POP_SIZE)]
best_fitness = -np.inf
best_individual = None

for gen in range(N_GEN):
    fitnesses = [fitness(ind) for ind in population]
    max_fit = max(fitnesses)
    print(f"Gen {gen}, best fitness: {-max_fit:.4f}")
    if max_fit > best_fitness:
        best_fitness = max_fit
        best_individual = population[np.argmax(fitnesses)]
    selected = select(population, fitnesses)
    new_pop = selected[:ELITE_SIZE]
    while len(new_pop) < POP_SIZE:
        p1, p2 = random.sample(selected, 2)
        child = crossover(p1, p2)
        if random.random() < MUTATION_RATE:
            child = mutate(child)
        new_pop.append(child)
    population = new_pop

# Evaluate best individual
qc_best = build_circuit(best_individual)
print(qc_best.draw(output='text'))
state_best = Statevector.from_instruction(qc_best)
probs_best = np.abs(state_best.data) ** 2
kl_best = kl_divergence(p_target, probs_best)

# Plot
labels = [f'{i:03b}' for i in range(len(p_target))]
x = np.arange(len(p_target))

plt.figure(figsize=(10, 5))
plt.bar(x - 0.15, p_target, width=0.3, label='Target', align='center')
plt.bar(x + 0.15, probs_best, width=0.3, label='Best Circuit', align='center')
plt.xticks(x, labels)
plt.xlabel('Basis State')
plt.ylabel('Probability')
plt.title(f'Final KL Divergence: {kl_best:.5f}')
plt.legend()
plt.tight_layout()
plt.show()

print("Best circuit:\n", qc_best.draw())


TypeError: QuantumCircuit.cz() missing 1 required positional argument: 'target_qubit'